In [1]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
from tabulate import tabulate

# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential1.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### Set date

In [31]:
date = '2020-11-03'
start = '2020-11-03 00:00:00'
end = '2020-11-04 00:00:00'

In [188]:
# df = pd.read_csv("./data/{}.csv".format(date))
# print("{}.csv".format(date))

#### Collect tweets with hashtag `#30mapchallenges`

In [15]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet':retweet_count,
                        'favorite':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [32]:
search_words = "#30DayMapChallenge -filter:retweets"
since = "2020-10-30"

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= start) & 
   (df.tweet_created_at <= end)]

# df1 = df[df['tweet_text'].str.contains("points") | df['tweet_text'].str.contains("day 1")]

df = df[df['media'].notnull()]
df['tweet_url'] = df['tweet_text'].str.extract('(?P<url>https?://[^\s]+)', expand=False).str.strip()


df.to_csv("./data/{}.csv".format(date))

Rate limit reached. Sleeping for: 121


In [33]:
df

,tweet_id,tweet_text,language,tweet_created_at,retweet,favorite,hashtags,user_id,user_name,user,user_loc,user_follower,friends_count,media,tweet_url
164,1323776467025756162,"#30DayMapChallenge Día3. Dada una distancia, ...",es,2020-11-03 23:57:58,3,9,"[{'text': '30DayMapChallenge', 'indices': [0, ...",28714934,Alonso Cortés,aloCrts,Mexico DF,122,411,http://pbs.twimg.com/media/El79jHnXYAAL6EF.jpg,https://t.co/dUaqwr15h9
165,1323776158807269377,Day 3 #30DayMapChallenge : Poligons\nMap: Polí...,es,2020-11-03 23:56:45,1,4,"[{'text': '30DayMapChallenge', 'indices': [6, ...",332567823,∑∂gαr∂ ∫. Grαnα∂0∫,Edgards04,"San Salvador, El Salvador",956,981,http://pbs.twimg.com/media/El7-tsCWoAAYyEx.jpg,https://t.co/N9soCjvDOZ
166,1323775135002685440,"This is a sliver of my home city of Lodz, PL u...",en,2020-11-03 23:52:41,2,8,"[{'text': '30DayMapChallenge', 'indices': [257...",1513661708,Andy Rutkowski,andy_rutkowski,,550,1388,http://pbs.twimg.com/media/El7-RiyU0AAWqoO.jpg,https://t.co/eJ5BXo3zfH
167,1323773467406184448,"#30DayMapChallenge Day 3, Polygon. Algo tarde ...",es,2020-11-03 23:46:03,3,15,"[{'text': '30DayMapChallenge', 'indices': [0, ...",384907052,Leticia Luna,Letyludigital,México,1121,2447,http://pbs.twimg.com/media/El78W8lU4AEnMV6.jpg,https://t.co/KX5vUCsQE0
168,1323771999202758656,"#30DayMapChallenge, day 3: Polygons - #Belize'...",en,2020-11-03 23:40:13,2,2,"[{'text': '30DayMapChallenge', 'indices': [0, ...",731808469301493760,🇧🇿 Belize GEO 🛰️,BZgeo,Belize,4116,4349,http://pbs.twimg.com/media/El768z-WkAEte_a.png,https://t.co/DqZ2Q0gGAO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,1323422517210583041,En el día 2 (líneas) del #30DayMapChallenge de...,es,2020-11-03 00:31:30,1,11,"[{'text': '30DayMapChallenge', 'indices': [25,...",43331985,Gonzalo Talavera Forlin,gonzalotalavera,"Lima, Perú",1503,837,http://pbs.twimg.com/media/El29QVcXgAEfp3v.png,https://t.co/AMOFPT7LtA
633,1323419075008106496,Day 3 of #30DayMapChallenge - Polygons\n\nVoro...,en,2020-11-03 00:17:50,1,0,"[{'text': '30DayMapChallenge', 'indices': [9, ...",47001569,ぞあ.,k_zoar,ジオイドと地表の狭間,828,751,http://pbs.twimg.com/media/El24WX0U8AA21FH.jpg,https://t.co/4GmKEQNjfW
634,1323417831656976385,Day 3 of #30DayMapChallenge - Polygons\n\nIrri...,en,2020-11-03 00:12:53,5,53,"[{'text': '30DayMapChallenge', 'indices': [9, ...",93839237,Andrew DC,andrewdc_nz,"Christchurch City, New Zealand",404,165,http://pbs.twimg.com/media/El25ZwTU4AEpXh5.jpg,https://t.co/jDLNYy4Nn0
635,1323416420500135936,In between my 14 hours of election volunteerin...,en,2020-11-03 00:07:17,2,20,"[{'text': '30DayMapChallenge', 'indices': [134...",2832470440,Katie Jolly,katiejolly6,"Seattle, WA",1099,1651,http://pbs.twimg.com/media/El23n1eU0AA8swb.jpg,https://t.co/wiscTcmzhY


### late entry (only for day 2)

In [7]:
# df[df['tweet_text'].str.contains('points', case=False) & 
#    df['tweet_text'].str.contains('day 1', case=False)].to_csv("./data/{}_late.csv".format(date))

In [8]:
# df = df.drop(df[df['tweet_text'].str.contains('points', case=False) & 
#                 df['tweet_text'].str.contains('day 1', case=False)].index)

---

## Overview

In [16]:
df[['retweet','favorite', 'user_follower', 'friends_count']].describe()

,retweet,favorite,user_follower,friends_count
count,393.000000,393.000000,393.000000,393.000000
mean,3.249364,16.778626,1409.720102,662.134860
std,5.836978,36.911358,5689.031444,857.301559
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,4.000000,77.000000,158.000000
50%,2.000000,8.000000,280.000000,335.000000
75%,4.000000,18.000000,876.000000,780.000000
max,91.000000,624.000000,80333.000000,5454.000000


## Top favorited `#30mapchallenges` post

In [34]:
df_fav15 = df.sort_values(by='favorite', ascending=False).head(15)
df_fav15 = df_fav15[['user', 'retweet', 'favorite', 'media']]
df_fav15['user'] = '**[@' + df['user'] +']('+ df['tweet_url']+')**'
df_fav15['media'] = '![](' + df['media'] +')'

In [35]:
print(tabulate(df_fav15, headers='keys', tablefmt='github'))

|     | user                                          |   retweet |   favorite | media                                                                                        |
|-----|-----------------------------------------------|-----------|------------|----------------------------------------------------------------------------------------------|
| 571 | **[@tjukanov](https://t.co/uLN9PJeg1k)**      |        21 |        239 | ![](http://pbs.twimg.com/media/El4Qv6tW0AEoreL.jpg)                                          |
| 371 | **[@Helenmckmck](https://t.co/pPbs6Py9ZI)**   |        21 |        153 | ![](http://pbs.twimg.com/media/El5-yUKWkAIS7OD.jpg)                                          |
| 538 | **[@kennethfield](https://t.co/O5is4PtFCb)**  |        17 |        138 | ![](http://pbs.twimg.com/media/El285DrUcAA_NEv.jpg)                                          |
| 506 | **[@dr_xeo](https://t.co/uUujVgV2ql)**        |        25 |         92 | ![](http://pbs.twimg.com/media/El4w90wX

## Top retweeted `#30mapchallenges` post

In [36]:
df_rt15 = df.sort_values(by='retweet', ascending=False).head(15)
df_rt15 = df_rt15[['user', 'retweet', 'favorite', 'media']]
df_rt15['user'] = '**[@' + df['user'] +']('+ df['tweet_url']+')**'
df_rt15['media'] = '![](' + df['media'] +')'

In [37]:
print(tabulate(df_rt15, headers='keys', tablefmt='github'))

|     | user                                            |   retweet |   favorite | media                                                                                        |
|-----|-------------------------------------------------|-----------|------------|----------------------------------------------------------------------------------------------|
| 506 | **[@dr_xeo](https://t.co/uUujVgV2ql)**          |        25 |         92 | ![](http://pbs.twimg.com/media/El4w90wXEAAu2gp.jpg)                                          |
| 571 | **[@tjukanov](https://t.co/uLN9PJeg1k)**        |        21 |        239 | ![](http://pbs.twimg.com/media/El4Qv6tW0AEoreL.jpg)                                          |
| 371 | **[@Helenmckmck](https://t.co/pPbs6Py9ZI)**     |        21 |        153 | ![](http://pbs.twimg.com/media/El5-yUKWkAIS7OD.jpg)                                          |
| 191 | **[@moaimx](https://t.co/aSJFhEp7za)**          |        18 |         80 | ![](http://pbs.twimg.com/me

### Late entry

In [25]:
# df_late = pd.read_csv("./data/{}_late.csv".format(date))
# print("{}_late.csv".format(date))
# print("Add to day 1...")

2020-11-02_late.csv
Add to day 1...


In [27]:
# df_late_fav = df_late.sort_values(by='favorite', ascending=False).head(5)
# df_late_fav = df_late_fav[['user', 'retweet', 'favorite', 'media']]
# df_late_fav['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_fav['media'] = '![](' + df_late['media'] +')'

In [28]:
print(tabulate(df_late_fav, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 42 | **[@darenw](https://t.co/rZ2gtbJmI4)**          |         4 |         95 | ![](http://pbs.twimg.com/media/Elyc4d4XUAEs09h.jpg) |
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 53 | **[@PratapVardhan](https://t.co/WTT8WM17qy)**   |         4 |         43 | ![](http://pbs.twimg.com/media/Elx5722WkAEOOMf.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 54 | **[@BZgeo](https://t.co/LavH2067O2)**           |         3 |         26 | ![](http://pbs.twimg.com/media/Elx0-fPXIAc9JGJ.png) |


In [29]:
# df_late_rt = df_late.sort_values(by='retweet', ascending=False).head(5)
# df_late_rt = df_late_rt[['user', 'retweet', 'favorite', 'media']]
# df_late_rt['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_rt['media'] = '![](' + df_late['media'] +')'

In [30]:
# print(tabulate(df_late_rt, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 24 | **[@neocartocnrs](https://t.co/1GcTGraT9T)**    |         8 |         14 | ![](http://pbs.twimg.com/media/Elz2OvNXEAAItfe.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 57 | **[@nurussadad](https://t.co/z4HaIfQMBi)**      |         4 |          7 | ![](http://pbs.twimg.com/media/ElxuzgpU0AEIbdT.png) |
|  7 | **[@geologyjesse](https://t.co/dR68mE4zhD)**    |         4 |         14 | ![](http://pbs.twimg.com/media/El1vroKWoAEozEk.jpg) |


### Send tweet via API

In [3]:
api.update_status("Look, I'm tweeting from #Python")


Status(_api=<tweepy.api.API object at 0x0000011E60202A48>, _json={'created_at': 'Mon Nov 02 22:53:36 +0000 2020', 'id': 1323397878983856130, 'id_str': '1323397878983856130', 'text': "Look, I'm tweeting from #Python", 'truncated': False, 'entities': {'hashtags': [{'text': 'Python', 'indices': [24, 31]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://hn303.github.io/CamLandEc-RM03/" rel="nofollow">Lonon_REST</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 528087151, 'id_str': '528087151', 'name': 'Haifeng Niu', 'screen_name': 'niu_haifeng', 'location': 'Cambridge, England', 'description': 'PhD candidate in Lab of Interdisciplinary Spatial Analysis, Department of Land Economy, University of Cambridge @CambridgeLandEc', 'url': 'https://t.co/nuaPhizg9Y', 'entities': {'url': {'urls': [{'url': 'https://t.co/nuaPhizg9Y', 'expanded_